In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
file_path = '/kaggle/input/heart-attack-dataset/heart_attack_dataset.csv'
df = pd.read_csv(file_path)
# Show the 
df.head(), df.columns

In [ ]:
# check for missing Values
df.info()


In [ ]:
# Summary of the dataset
df.describe()

In [ ]:
# Missing Vakues
#Handle missing values if necessary

df = df.fillna(df.mean(numeric_only=True))

In [ ]:
#Verify no missing value remaining
df.isnull().sum()


In [ ]:
# Convert categorical columns to numeric
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
df['Has Diabetes'] = df['Has Diabetes'].map({'Yes': 1, 'No': 0})
df['Smoking Status'] = df['Smoking Status'].map({'Smoker': 1, 'Non-Smoker': 0})
df['Chest Pain Type'] = df['Chest Pain Type'].astype('category').cat.codes

# Check the updated dataframe
df.head()

In [ ]:
# Matplotlib for  graphs

import matplotlib.pyplot as plt

import seaborn as sns

# Set the aesthetic style of the plots
sns.set(style="whitegrid")

# Distribution of Age
plt.figure(figsize=(10, 6))
sns.histplot(df['Age'], kde=True, color='blue')
plt.title('Distribution of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

